# Riksdagen corpus v0.14.0

2024-02-26

This Colab notebook demonstrates how to quickly access data from the Riksdagen corpus.

Let's install them and set things up

In [1]:
#%pip install pyriksdagen
from lxml import etree
import progressbar
from pyparlaclarin.read import paragraph_iterator, speeches_with_name
from pyriksdagen.utils import protocol_iterators, download_corpus
import pyriksdagen
# We need a parser for reading in XML data
parser = etree.XMLParser(remove_blank_text=True)

Then, let's download the corpus

In [2]:
print(pyriksdagen.__spec__)
download_corpus(partitions=["politicians", "records"])

ModuleSpec(name='pyriksdagen', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1039f1420>, origin='/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.10/site-packages/pyriksdagen/__init__.py', submodule_search_locations=['/opt/homebrew/Caskroom/miniforge/base/envs/data/lib/python3.10/site-packages/pyriksdagen'])


politicians.zip: 100%|██████████| 2.20M/2.20M [00:00<00:00, 36.3MiB/s]

15:12:05 [WARNING] (pyriksdagen): data already exists at the path 'data'. It will be overwritten once the download is finished.



records.zip: 100%|██████████| 1.60G/1.60G [00:42<00:00, 40.0MiB/s]


OSError: [Errno 28] No space left on device

Now we can go over some protocols from, say, 1955-1956.

In [4]:
protocols = list(protocol_iterators(corpus_root="data/", start=1955, end=1956))
protocols[:5]

['data/1956/prot-1956--ak--001.xml',
 'data/1956/prot-1956--ak--002.xml',
 'data/1956/prot-1956--ak--003.xml',
 'data/1956/prot-1956--ak--004.xml',
 'data/1956/prot-1956--ak--005.xml']

It is straightforward to print out all content, including speeches, dates, speaker introductions and topic titles.

In [5]:
# Select a protocol cause it's a whole lot of text
protocol_in_question = protocols[12]
print(protocol_in_question)
root = etree.parse(protocol_in_question, parser).getroot()

data/1956/prot-1956--ak--013.xml


In [6]:
for elem in list(paragraph_iterator(root, output="lxml"))[:7]:
  print(" ".join(elem.itertext()))



          RIKSDAGENS PROTOKOLL
        

          1956 ANDRA KAMMAREN Nr 13
        

          10—11 april
        

          Debatter m. m.
        

          Tisdagen den 10 april Sid. Svar på interpellationer av: herr
          Norén ang. tillämpningen av bestämmelserna rörande egnahemsbebyggelsen
          på landsbygden .........s.ss=>ssessseecrreersereerert 5 fröken
          Elmén ang. samordningen mellan arbetslöshetsförsäkring och sjukförsäkring
          ssssssseseseereereerereerr rer rr rr RR RR RR RAR 9 herr Nilsson
          i Bästekille i anledning av att de nya vägarna mellan Hässleholm
          och Malmö och mellan Gärsnäs och Tomelilla dragas fram över värdefull
          åkerjord ..........s.sss-sesrseresesrtorsn orter ra 12 herr Östlund
          ang. ökad kvot till Västerbottens län för investering i vägoch
          brobyggnader...........ssesseresereerserserers rr ers et 15
        

          Interpellationer av: herr Lundberg ang. faran för förstöring
   

Moreover, the metadata catalogues are also available. They are stored as CSV files, and can be accessed with pandas in python, or the spreadsheet program of your choice.

In [11]:
from pyriksdagen.metadata import load_Corpus_metadata

df = load_Corpus_metadata("data")

                      person_id       start   end  \
0      i-PFAPNmRqeUAaxDzNRTG1x1        1867  1867   
1      i-QSYHiJ6G54WwZYYDpVnD4u        1867  1867   
2      i-Ddmtm1uG9esPH37c8XjUXZ        1867  1867   
3      i-65rmwEXkkhA1kxSrD4oMUw        1867  1867   
4      i-AvGpsGJvs5PXTcEG4DtbFt        1867  1867   
...                         ...         ...   ...   
13206  i-DhFtpnk9dqFrTXA2tvjJnb  2024-01-30   NaN   
13207  i-9rzEiT5qs2rumTNiqr6kVD  2024-01-31   NaN   
13208  i-Swns2VJiq8mNPgw8cBRfJ9  2024-02-12   NaN   
13209   i-x1CuoKmRHYgQr9i2kh3B5         NaN   NaN   
13210  i-TUyWWYGDFXW92GhiG3CLwF         NaN   NaN   

                                    district                 role  
0          Eskilstuna och Strängnäs valkrets   andrakammarledamot  
1      Västra Götalands läns västra valkrets  förstakammarledamot  
2                      Torna härads valkrets   andrakammarledamot  
3                    Värmlands läns valkrets  förstakammarledamot  
4                Kristi

FileNotFoundError: [Errno 2] No such file or directory: 'data/riksdag_start-end.csv'

Let's find a specific person based on their name, for example Elis Håstad

In [12]:
# Let's find mr. Håstad
df[df.name.str.contains("Håstad")]

NameError: name 'df' is not defined

His identifier is i-S9dMiL9yRpDaYPfVGjK1Gp. Using that, we can find all his speeches. Let's do that and print out the first one

In [8]:
# Elis Håstad (i-S9dMiL9yRpDaYPfVGjK1Gp)
hastad_speeches = []
for protocol in progressbar.progressbar(protocols):
  root = etree.parse(protocol, parser).getroot()
  protocol_speech = []
  for speech in speeches_with_name(root, name="i-S9dMiL9yRpDaYPfVGjK1Gp"):
    protocol_speech.append(speech)
  protocol_speech = "\n".join(protocol_speech).strip()
  if protocol_speech != "":
    hastad_speeches.append(protocol_speech)

print(hastad_speeches[0])

100% (130 of 130) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


Herr talman! Då debatten är så långt framskriden, skall jag inte
            gå in på hela detta stora ämne. Jag kan i allt väsentligt instämma
            i de rent praktiska synpunkter, som herr Fast här har givit till
            känna. Jag skall inte heller gå in på frågan om det stora intrång
            på den kyrkliga självstyrelsen, som motionen evident syftar till.
          

            Vad jag skulle vilja begränsa mig till är frågan, huruvida herr
            Lundberg och herr Edberg och övriga motionärer i denna kammare,
            när de nu kräver att en stor del av beslutanderätten på det kyrkliga
            området skall överflyttas till den borgerliga kommunen, företräder
            en mera demokratisk åsikt än anhängarna av den nu gällande ordningen.
            Det har väl allmänt betraktats som en stor vinning, att vi 1862
            gjorde en åtskillnad mellan den borgerliga och den kyrkliga kommunen.
            Denna vinning har ju bestått sedan man på bägge